# CSCI B505 – Fall 2019
## Programming Assignment 5:

### Dong Liang

In [478]:
import numpy as np
# import re
# import heapq

# from toolkits.util import *

# %load_ext autoreload
# %autoreload 2

## Huffman's algorithm implementation

In [479]:
class Node(object):  
    def __init__(self, char, freq):
        self.left = None
        self.right = None
        self.parent = None
        self.char = char
        self.freq = freq       
        
    @property
    def is_root(self):
    	return self.parent == None

    @property
    def is_left(self):
    	if not self.is_root:
        	return self.parent.left == self
    	else:
    		return False

    @property
    def is_leaf(self):
        return (self.left == None) and (self.right == None)



class Huffman(object):
	def __init__(self, freq_dict):
		self.freq_dict = freq_dict
		self.leaves = [Node(char, freq) for char, freq in self.freq_dict.items()]
		self.Huffman_tree = None


	def _build_Huffman_tree(self):   
	    queue = self.leaves.copy()
	    Huffman_tree = []
	    
	    while len(queue) > 1:
	        queue.sort(key = lambda node: node.freq)

	        left = queue.pop(0)         
	        right = queue.pop(0)          
	        
	        parent = Node(left.char + right.char, left.freq + right.freq)   
	        parent.left = left                     
	        parent.right = right                    
	        
	        left.parent = parent         
	        right.parent = parent        
	        
	        queue.append(parent)

	        Huffman_tree.append(parent)   

	    Huffman_tree.extend(self.leaves)
	    self.Huffman_tree = Huffman_tree

	    return self.Huffman_tree 


	def get_Huffman_code(self):

		self._build_Huffman_tree()

		nodes_leaf = [node for node in self.Huffman_tree if node.is_leaf]
		
		Huffman_codes = {}

		for leaf in nodes_leaf:
			current_node = leaf	
			code = ''
			while not current_node.is_root:
				if current_node.is_left:
					code = '0' + code 
				else: 
					code = '1' + code
				current_node = current_node.parent
			Huffman_codes[leaf.char] = code	

		return Huffman_codes


	def encoding(self, text):
		
		Huffman_codes = self.get_Huffman_code()
		Huffman_text = ''
		
		for char in text:
			Huffman_text = Huffman_text + Huffman_codes[char]

		return Huffman_text		

	
	def decoding(self, Huffman_text):
		self._build_Huffman_tree()

		text = ''
		root = self.Huffman_tree[np.argmax([t.is_root for t in self.Huffman_tree])]
		current_node = root
		
		for code in Huffman_text:
			code = int(code)
			
			if code == 0:
				current_node = current_node.left
			elif code == 1:
				current_node = current_node.right

			if current_node.is_leaf:
				text = text + current_node.char
				current_node = root

		return text


## Data preprocessing

In [480]:
def split(word): 
    return [char for char in word]  

def is_feasible(char):
    return char in "abcdefghijklmnopqrstuvwxyz .,!?'"


def convert(file_name):
    freq_dict = {}
    with open(file_name, 'r') as file:
        lines = file.readlines()
        text = ''
        for line in lines:

            line = line.strip()
            line = line.lower()

            for char in split(line):
                if is_feasible(char):
                    text = text + char
                    
                    if char not in freq_dict.keys():
                        freq_dict[char] = 1
                    else:
                        freq_dict[char] += 1
    
    return text, freq_dict

Let's first load and convert the original text to the long string with only 32 characters (`text`). At the same, we will also compute the character freqency table and store it in a varaible called `freq_dict`

In [481]:
text, freq_dict = convert('book.txt')

In [490]:
# Print first 1000 string of the original text file
text[:1000]

"the project gutenberg ebook of women of belgium turning tragedy to triumph, bycharlotte kelloggthis ebook is for the use of anyone anywhere in the united states andmost other parts of the world at no cost and with almost no restrictionswhatsoever.  you may copy it, give it away or reuse it under the termsof the project gutenberg license included with this ebook or online atwww.gutenberg.org.  if you are not located in the united states, you'llhave to check the laws of the country where you are located before usingthis ebook.title women of belgium turning tragedy to triumphauthor charlotte kelloggrelease date october ,  ebook language englishcharacter set encoding utf start of this project gutenberg ebook women of belgium produced by f e h, mws and the online distributedproofreading team at httpwww.pgdp.nettranscribers notechanges made are noted at the end of the book.illustration a little bees diningroom for subnormal childrenwomen of belgiumturning tragedy to triumphbycharlotte kello

In [489]:
# Print the frequency table
freq_dict

{'t': 14963,
 'h': 9331,
 'e': 20817,
 ' ': 32533,
 'p': 2842,
 'r': 10278,
 'o': 12232,
 'j': 167,
 'c': 4275,
 'g': 3491,
 'u': 4203,
 'n': 10845,
 'b': 2731,
 'k': 1081,
 'f': 4052,
 'w': 3487,
 'm': 3948,
 'l': 6225,
 'i': 10706,
 'a': 11839,
 'd': 6140,
 'y': 2836,
 ',': 2418,
 's': 9570,
 'v': 1494,
 '.': 1625,
 "'": 9,
 'x': 293,
 'z': 123,
 'q': 175,
 '?': 39,
 '!': 159}

## Huffman encoding and decoding

We then construct a Huffman instance and the text string will be encoded and decoded using Huffman codes and Huffman tree, respectively

In [482]:
# Build a Huffman instance
huffman = Huffman(freq_dict)

# Get Huffman codes for all the 32 characters
huffman.get_Huffman_code()

{'t': '1100',
 'h': '0000',
 'e': '010',
 ' ': '111',
 'p': '100100',
 'r': '0011',
 'o': '1010',
 'j': '0010000111',
 'c': '110111',
 'g': '101111',
 'u': '110110',
 'n': '0111',
 'b': '001010',
 'k': '0010001',
 'f': '110101',
 'w': '101110',
 'm': '110100',
 'l': '10110',
 'i': '0110',
 'a': '1000',
 'd': '10011',
 'y': '001011',
 ',': '001001',
 's': '0001',
 'v': '1001010',
 '.': '1001011',
 "'": '00100000000',
 'x': '001000010',
 'z': '0010000001',
 'q': '001000001',
 '?': '00100000001',
 '!': '0010000110'}

In [483]:
# Get the Huffman-encoded text
text_encoded = huffman.encoding(text)

# Print the first 100 strings
text_encoded[:1000]

'110000000101111001000011101000100001110101101111100111101111110110110001001110010100100011101111111010001010101010100010001111101011010111110111010101101000100111111101011010111100101001010110101111011011011011010011111001101100011011101100111101111111110000111000101111010100110010111111100101011111000011011011011011010010010000000010011110010100010111101110000100000111011010101100110001011100100010101011010110101010111110111111000000011000011110100010101010101000100011110110000111111010110100011111110000000101111101100001010111101011010111110000111001011101001110101111000011100101110111000000100011010111011001111111100000001011111011001110110110001010011111000111001000110001000011111000011110011110100101000011100111101011000000010001111110010010000011110000011111010110101111110000000101111011101010001110110100111111000110011101111010111110111101000011100111100001111001111110111001101100000011110001011011010010100001110011101111010111001101000011100001101101101111100011010100111000110

In [484]:
# Recover the original string (text) using Huffman-encoded text
text_decoded = huffman.decoding(text_encoded)

# Print the first 1000 strings of the decoded text
text_decoded[:1000]

"the project gutenberg ebook of women of belgium turning tragedy to triumph, bycharlotte kelloggthis ebook is for the use of anyone anywhere in the united states andmost other parts of the world at no cost and with almost no restrictionswhatsoever.  you may copy it, give it away or reuse it under the termsof the project gutenberg license included with this ebook or online atwww.gutenberg.org.  if you are not located in the united states, you'llhave to check the laws of the country where you are located before usingthis ebook.title women of belgium turning tragedy to triumphauthor charlotte kelloggrelease date october ,  ebook language englishcharacter set encoding utf start of this project gutenberg ebook women of belgium produced by f e h, mws and the online distributedproofreading team at httpwww.pgdp.nettranscribers notechanges made are noted at the end of the book.illustration a little bees diningroom for subnormal childrenwomen of belgiumturning tragedy to triumphbycharlotte kello

## Calculate the bits saved by using Huffman encoding

In [485]:
# Before Huffman encoding
num_of_char = np.sum(list(freq_dict.values()))
bits = 5 * num_of_char
bits

974635

In [486]:
# After Huffman encoding
bits_huffman = len(text_encoded) 
bits_huffman

825259

In [487]:
# Bits saved by Huffman encoding
Diff = bits - bits_huffman
Diff

149376

In [488]:
# Percent of bits saved by Huffman encoding
Percent = 100.0 * (1. - bits_huffman / bits)
Percent

15.326352942383558